# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 22 2022 9:30AM,242531,20,ALUR089923187,Alumier Labs Inc.,Released
1,Jul 22 2022 9:30AM,242531,20,ALUR296914219,Alumier Labs Inc.,Released
2,Jul 22 2022 9:30AM,242531,20,ALUR307738791,Alumier Labs Inc.,Released
3,Jul 22 2022 9:30AM,242531,20,ALUR317544690,Alumier Labs Inc.,Released
4,Jul 22 2022 9:30AM,242531,20,ALUR383727930,Alumier Labs Inc.,Released
5,Jul 22 2022 9:30AM,242531,20,ALUR413852008,Alumier Labs Inc.,Released
6,Jul 22 2022 9:30AM,242531,20,ALUR535808439,Alumier Labs Inc.,Released
7,Jul 22 2022 9:30AM,242531,20,ALUR900894927,Alumier Labs Inc.,Released
8,Jul 22 2022 9:30AM,242531,20,ALUR936758463,Alumier Labs Inc.,Released
9,Jul 22 2022 9:30AM,242531,20,ALUR493892621,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,242525,Released,13
39,242528,Released,1
40,242529,Released,1
41,242530,Released,1
42,242531,Released,12


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242525,NaN,7.0,13.0
242528,NaN,NaN,1.0
242529,NaN,NaN,1.0
242530,NaN,NaN,1.0
242531,NaN,NaN,12.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242348,47.0,15.0,14.0
242411,0.0,4.0,0.0
242427,0.0,0.0,1.0
242431,0.0,1.0,0.0
242439,0.0,7.0,6.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242348,47,15,14
242411,0,4,0
242427,0,0,1
242431,0,1,0
242439,0,7,6


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242348,47,15,14
1,242411,0,4,0
2,242427,0,0,1
3,242431,0,1,0
4,242439,0,7,6


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242348,47,15,14
1,242411,,4,
2,242427,,,1
3,242431,,1,
4,242439,,7,6


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 22 2022 9:30AM,242531,20,Alumier Labs Inc.
12,Jul 22 2022 9:23AM,242525,15,"Alliance Pharma, Inc."
32,Jul 22 2022 9:22AM,242530,10,"Citieffe, Inc."
33,Jul 22 2022 9:22AM,242520,15,"Virtus Pharmaceuticals, LLC"
38,Jul 22 2022 9:22AM,242529,10,Ivaoes Animal Health
39,Jul 22 2022 9:22AM,242528,10,"Nextsource Biotechnology, LLC"
40,Jul 22 2022 9:17AM,242522,10,ISDIN Corporation
41,Jul 22 2022 9:14AM,242521,10,ISDIN Corporation
54,Jul 22 2022 9:08AM,242517,10,"Senseonics, Incorporated"
55,Jul 22 2022 9:06AM,242514,16,Sartorius Stedim North America


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 22 2022 9:30AM,242531,20,Alumier Labs Inc.,,,12
1,Jul 22 2022 9:23AM,242525,15,"Alliance Pharma, Inc.",,7,13
2,Jul 22 2022 9:22AM,242530,10,"Citieffe, Inc.",,,1
3,Jul 22 2022 9:22AM,242520,15,"Virtus Pharmaceuticals, LLC",,2,3
4,Jul 22 2022 9:22AM,242529,10,Ivaoes Animal Health,,,1
5,Jul 22 2022 9:22AM,242528,10,"Nextsource Biotechnology, LLC",,,1
6,Jul 22 2022 9:17AM,242522,10,ISDIN Corporation,,,1
7,Jul 22 2022 9:14AM,242521,10,ISDIN Corporation,,,13
8,Jul 22 2022 9:08AM,242517,10,"Senseonics, Incorporated",,,1
9,Jul 22 2022 9:06AM,242514,16,Sartorius Stedim North America,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 22 2022 9:30AM,242531,20,Alumier Labs Inc.,12,,
1,Jul 22 2022 9:23AM,242525,15,"Alliance Pharma, Inc.",13,7,
2,Jul 22 2022 9:22AM,242530,10,"Citieffe, Inc.",1,,
3,Jul 22 2022 9:22AM,242520,15,"Virtus Pharmaceuticals, LLC",3,2,
4,Jul 22 2022 9:22AM,242529,10,Ivaoes Animal Health,1,,
5,Jul 22 2022 9:22AM,242528,10,"Nextsource Biotechnology, LLC",1,,
6,Jul 22 2022 9:17AM,242522,10,ISDIN Corporation,1,,
7,Jul 22 2022 9:14AM,242521,10,ISDIN Corporation,13,,
8,Jul 22 2022 9:08AM,242517,10,"Senseonics, Incorporated",1,,
9,Jul 22 2022 9:06AM,242514,16,Sartorius Stedim North America,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 22 2022 9:30AM,242531,20,Alumier Labs Inc.,12,,
1,Jul 22 2022 9:23AM,242525,15,"Alliance Pharma, Inc.",13,7,
2,Jul 22 2022 9:22AM,242530,10,"Citieffe, Inc.",1,,
3,Jul 22 2022 9:22AM,242520,15,"Virtus Pharmaceuticals, LLC",3,2,
4,Jul 22 2022 9:22AM,242529,10,Ivaoes Animal Health,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 22 2022 9:30AM,242531,20,Alumier Labs Inc.,12.0,NaN,NaN
1,Jul 22 2022 9:23AM,242525,15,"Alliance Pharma, Inc.",13.0,7.0,NaN
2,Jul 22 2022 9:22AM,242530,10,"Citieffe, Inc.",1.0,NaN,NaN
3,Jul 22 2022 9:22AM,242520,15,"Virtus Pharmaceuticals, LLC",3.0,2.0,NaN
4,Jul 22 2022 9:22AM,242529,10,Ivaoes Animal Health,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 22 2022 9:30AM,242531,20,Alumier Labs Inc.,12.0,0.0,0.0
1,Jul 22 2022 9:23AM,242525,15,"Alliance Pharma, Inc.",13.0,7.0,0.0
2,Jul 22 2022 9:22AM,242530,10,"Citieffe, Inc.",1.0,0.0,0.0
3,Jul 22 2022 9:22AM,242520,15,"Virtus Pharmaceuticals, LLC",3.0,2.0,0.0
4,Jul 22 2022 9:22AM,242529,10,Ivaoes Animal Health,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3637577,184.0,8.0,0.0
12,484957,1.0,14.0,0.0
15,485045,16.0,9.0,0.0
16,727423,2.0,4.0,0.0
19,969979,10.0,23.0,0.0
20,969768,36.0,22.0,47.0
21,727322,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3637577,184.0,8.0,0.0
1,12,484957,1.0,14.0,0.0
2,15,485045,16.0,9.0,0.0
3,16,727423,2.0,4.0,0.0
4,19,969979,10.0,23.0,0.0
5,20,969768,36.0,22.0,47.0
6,21,727322,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,184.0,8.0,0.0
1,12,1.0,14.0,0.0
2,15,16.0,9.0,0.0
3,16,2.0,4.0,0.0
4,19,10.0,23.0,0.0
5,20,36.0,22.0,47.0
6,21,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,184.0
1,12,Released,1.0
2,15,Released,16.0
3,16,Released,2.0
4,19,Released,10.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,47.0,0.0
Executing,8.0,14.0,9.0,4.0,23.0,22.0,1.0
Released,184.0,1.0,16.0,2.0,10.0,36.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,47.0,0.0
1,Executing,8.0,14.0,9.0,4.0,23.0,22.0,1.0
2,Released,184.0,1.0,16.0,2.0,10.0,36.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,47.0,0.0
1,Executing,8.0,14.0,9.0,4.0,23.0,22.0,1.0
2,Released,184.0,1.0,16.0,2.0,10.0,36.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()